## Trajectory classification with Gesture Variation Follower

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns

t1 = np.sin(5. * np.pi * np.linspace(0, 0.5, 20)) * 0.5 + 0.5
t2 = np.sin(2. * np.pi * np.linspace(0, 2, 15)) * 0.5 + 0.5
plt.plot(t1 + np.random.normal(scale=0.05, size=t1.size))
plt.plot(t2 + np.random.normal(scale=0.05, size=t2.size))

In [ ]:
import numpy as np


class GVF(object):
    def __init__(self, ns, sigs, icov, resThresh, nu=0.):
        """Constructor of GVF.

        ns          number of particles
        sigs        table of sigmas for each adapted parameters
        icov        static std dev for observation likelihood
        resThresh   threshold for resampling
        nu          t-dist parameter
        """
        self.pdim = sigs.shape[0]  # number of state dimensions

        self.X = np.zeros((ns, self.pdim))  # Each row is a particle
        self.g = np.zeros(ns, dtype=int)  # gesture index for each particle [g is ns x 1]
        self.w = np.zeros(ns)  # weight of each particle [w is ns x 1]
        self.logW = np.zeros(ns)  # non-normalized weights
        self.sigt = np.sqrt(sigs)  # vector of variances
        # Filled in in `spreadParticles`
        self.means = None  # vector of means for particles initial spreading
        self.ranges = None   # vector of ranges around the means for particles initial spreading
        self.R_multi = []  # gesture references (several examples)
        self.R_single = []  # gesture references (1 example)
        self.icov_single = icov  # inverse covariance (coeff. for the diagonal matrix)
        self.nu = nu  # degree of freedom for the t-distribution; if 0, use a Gaussian
        self.sp, self.sv, self.sr, self.ss = sigs # sigma values (actually, their square root)
        self.resampling_threshold = resThresh
        self.lrndGstr = -1  # number of learned gestures
        self.multivar = False  # whether to be multivariate? deprecated?
        self.gestureLengths = []  # length of each reference gesture

    def initweights(self):
        ns = self.w.size
        self.w[...] = 1. / ns

    def addTemplate(self):
        """Add a template to the database by allocating the memory.
    
        Needs to be called before the fillTemplate() method.
        """
        self.lrndGstr += 1
        self.R_single.append([])
        self.gestureLengths.append(0)

    def fillTemplate(self, id, data):
        """Fill the template given by the integer 'id'.

        With the current data vector 'data'
        """
        if id <= self.lrndGstr:
            self.R_single[id].append(data)
            self.gestureLengths[id] += 1

    def spreadParticles(self, meanPVRS, rangePVRS):
        """Spread particles by sampling values from given intervals.

        The current implemented distribution is uniform.

        meanPVRS    mean values around which the particles are sampled
        rangePVRS   range values defining how far from the means the particles can be spread
        """
        ns = self.X.shape[0]
        ngestures = self.lrndGstr+1

        # Keep track of means / ranges
        self.means = meanPVRS
        self.ranges = rangePVRS

        # Spread particles using a uniform distribution
        for i in range(self.pdim):
            for n in range(ns):
                self.X[n, i] = np.random.uniform(-0.5, 0.5) * rangePVRS[i] + meanPVRS[i]

        # Weights are also uniformly spread
        self.initweights()
        self.logW[:] = 0.0

        pass

    def particleFilter(self, obs):
        """Core algorithm: does one step of inference using particle filtering"""
        ns = self.X.shape[0]  # n_particles
        # Particles outside
        particle_before_0 = []
        particle_after_1 = []

        # Main loop: same process for each particle (row n in X)
        for n in range(ns):
            # Move the particle
            # Position respects a first order dynamic: p = p + v/L
            self.X[n, 0] = (self.X[n, 0]
                            + np.random.normal(1) * self.sigt[0]
                            + self.X[n, 1] / self.gestureLengths[self.g[n]])

            # Move the other state elements according to Gaussian noise
            for l in range(1, self.pdim):
                self.X[n, l] = self.X[n, l] + np.random.normal(1) * self.sigt[0]

            x_n = self.X[n]
            if x_n[0] < 0:
                # Can't observe a particle outside (0, 1) range [this behavior could be changed]
                self.w[n] = 0
                particle_before_0.append(n)
                self.logW[n] = -np.inf
            elif x_n[0] > 1:
                self.w[n] = 0
                particle_after_1.append(n)
                self.logW[n] = -np.inf
            else:
                pgi = self.g[n]  # gesture index
                frameindex = min(int(self.gestureLengths[pgi]-1),
                                 int(np.floor(x_n[0] * self.gestureLengths[pgi])))
                vref = np.array(self.R_single[pgi][frameindex])

                # If incoming data is 2-dimensional: we assume it is a drawn shape!
                if obs.size == 2:
                    # scaling
                    vref *= x_n[2]
                    # rotation
                    alpha = x_n[3]
                    rotmat = np.array([np.cos(alpha), -np.sin(alpha), np.sin(alpha), np.cos(alpha)])
                    vref = rotmat * vref  # or np.dot?
                elif obs.size == 3:
                    # scaling
                    vref *= x_n[2]

                # Observation likelihood and update weights
                dist = np.dot(vref - obs, vref - obs) * self.icov_single
                if self.nu == 0.:  # Gaussian distribution
                    self.w[n] *= np.exp(-dist)
                    self.logW[n] += -dist
                else:  # Student's distribution
                    # NB! dimension is 2
                    self.w[n] *= np.power(dist / self.nu + 1, -self.nu / 2 - 1)
                    self.logW[n] += (-self.nu / 2 - 1) * np.log(dist / nu + 1)
        
        # TODO: here we should compute the 'absolute likelihood' as log(w) before normalization
        # this absolute likelihood could be used as a raw criterion for segmentation
        for n in range(len(particle_before_0)):
            # Spread particles using a uniform distribution
            for i in range(self.pdim):
                self.X[particle_before_0[n], i] = np.random.uniform(-0.5, 0.5) * self.ranges[i] + self.means[i]
            self.w[particle_before_0[n]] = 1. / ns
            self.g[particle_before_0[n]] = n % (self.lrndGstr + 1)
        for n in range(len(particle_after_1)):
            # Spread particles using a uniform distribution
            for i in range(self.pdim):
                self.X[particle_after_1[n], i] = np.random.uniform(-0.5, 0.5) * self.ranges[i] + self.means[i]
            self.w[particle_after_1[n]] = 1. / ns
            self.g[particle_after_1[n]] = n % (self.lrndGstr + 1)

        # Normalization - resampling
        self.w /= self.w.sum()
        neff = 1. / np.dot(self.w, self.w)
        if neff < self.resampling_threshold:
            self.resampleAccordingToWeights()
            self.initweights()

    def resampleAccordingToWeights(self):
        ns = self.w.shape[0]
        oldX = np.array(self.X)
        oldG = np.array(self.g)
        oldLogW = np.array(self.logW)
        c = np.zeros(ns)

        for i in range(1, ns):
            c[i] = c[i-1] + self.w[i]

        i = 0
        u0 = np.random.uniform(0, 1) / ns
        for j in range(ns):
            uj = u0 + (j + 0.) / ns
            while uj > c[i] and i < ns - 1:
                i += 1
            self.X[j] = oldX[i]
            self.g[j] = oldG[i]
            self.logW[j] = oldLogW[i]

    def infer(self, vect):
        """Run inference on the input dataset.

        Each row is a temporal observation.
        For each row the incremental inference procedure is called.

        vect     whole data matrix
        """
        self.particleFilter(vect)

    def getGestureConditionalProbabilities(self):
        """Return values of each gesture's likelihood"""
        ngestures = self.lrndGstr+1
        ns = self.X.shape[0]
        gp = np.zeros(ngestures)
        for n in range(ns):
            gp[self.g[n]] += self.w[n]
        return gp

    def getGestureLikelihoods(self):
        """Return values of each gesture's likelihood"""
        ngestures = self.lrndGstr+1
        ns = self.X.shape[0]
        numg = np.zeros(ngestures)
        gp = np.zeros(ngestures)
        for n in range(ns):
            if self.logW[n] > -np.inf:
                gp[self.g[n]] += self.logW[n]
                numg[self.g[n]] += 1
        for n in range(ngestures):
            if numg[n] == 0:
                gp[n] = -np.inf
            else:
                gp[n] = gp[n] / numg[n]
        return gp

    def getEndGestureProbabilities(self, minpos=0.):
        ngestures = self.lrndGstr+1
        ns = self.X.shape[0]
        gp = np.zeros(ngestures)
        for n in range(ns):
            if self.X[n, 0] > minpos:
                gp[self.g[n]] += self.w[n]
        return gp

    def getEstimatedStatus(self):
        """Return values of estimated features"""
        ngestures = self.lrndGstr+1
        ns = self.X.shape[0]
        es = np.zeros((ngestures, self.pdim+1))  # PVRSW

        for n in range(ns):
            gi = self.g[n]
            es[gi, :self.pdim] += self.X[n] * self.w[n]
            es[gi, self.pdim] += self.w[n]

        # Ensure we don't get nans
        es[es == 0] = np.finfo(float).eps
        for gi in range(ngestures):
            es[gi, :self.pdim] /= es[gi, self.pdim]

        return es

    def getResamplingThreshold(self):
        return self.resampling_threshold

    def getNbOfParticles(self):
        return self.w.size

    def getnbOfTemplates(self):
        return self.gestureLengths.size

    def getLengthOfTemplateByInd(self, ind):
        if ind < self.gestureLengths.size:
            return self.gestureLengths[ind]
        return -1

    def getTemplateByInd(self, ind):
        if ind < self.gestureLengths.size:
            return self.R_single[ind]
        else:
            return []

    def setIcovSingleValue(self, f):
        if f > 0:
            self.icov_single = f

    def setAdaptSpeed(self, speed):
        if speed.size == self.pdim:
            self.sigt = np.sqrt(speed)

    def setResamplingThreshold(self, r):
        self.resampling_threshold = r

    def clear(self):
        del self.R_single[:]
        del self.gestureLengths[:]
        self.lrndGstr = -1

In [ ]:
so = 0.2  # sigma observation?
gvf = GVF(ns=2000,
          sigs=np.array([0.0001, 0.01, 0.000001, 0.0001]),
          icov=1./(so * so),
          resThresh=1000,
          nu=0.)
mpvrs = np.array([0.05, 1.0, 1.0, 0.0])
rpvrs = np.array([0.1, 0.4, 0.3, 0.0])

# First, add some templates with 'addTemplate', 'fillTemplate'
gvf.addTemplate()
gvf.addTemplate()
# Give some noisy examples
for _ in range(2):
    gvf.fillTemplate(0, t1 + np.random.normal(scale=0.05, size=t1.size))
    gvf.fillTemplate(1, t2 + np.random.normal(scale=0.05, size=t2.size))

# Second, call 'spreadParticles' to start the particle filters
gvf.spreadParticles(mpvrs, rpvrs)

# Then, you can generate observations and call 'infer'
for i, obs in enumerate(t1):
    gvf.infer(obs)
    if i % 3 == 0:
        print(gvf.getEstimatedStatus())
        print(gvf.getGestureConditionalProbabilities())
        print(gvf.getGestureLikelihoods())